## **Simple neural network with Pytorch**

In [ ]:
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)

In [ ]:
import matplotlib.pyplot as plt
idx = 1000
plt.imshow(train_dataset.data[idx], cmap='gray')
plt.title(f'Class: {train_dataset.targets[idx]}')
plt.show()

In [ ]:
train_loader = DataLoader(train_dataset, **{'batch_size': 32})
test_loader = DataLoader(test_dataset, **{'batch_size': 32})

In [ ]:
class OneLayerNN(nn.Module):
    def __init__(self):
        super(OneLayerNN, self).__init__()
        # define layers here
        self.fc = nn.Linear(784, 10)  # fc => fully connected layer, 28 * 28 = 784
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
torch.manual_seed(126)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # check device
model = OneLayerNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # variant of gradient descent algorithm

In [ ]:
model.train()
n_epochs = 1  # change here
for epoch in range(1, n_epochs + 1):
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
def predict(input_data):
    logit = model(input_data.unsqueeze(-1))
    y_pred = F.softmax(logit, dim=1).argmax(dim=-1)
    return y_pred

In [ ]:
data, target = next(iter(test_loader))
idx = 3
sample, y = data[idx], target[idx]
plt.imshow(sample.squeeze(0), cmap='gray')
plt.title(f'Actual class: {y}, Predicted class: {predict(sample)[0]}')
plt.show()

In [ ]:
# predict all
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for data, target in tqdm(test_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        y_true.extend(target.tolist())
        y_pred.extend(pred.flatten().tolist())

In [ ]:
print("Accuracy", accuracy_score(y_true, y_pred))
print("Precision, Recall, F1-score", precision_recall_fscore_support(y_true, y_pred, average="macro"))

## **Adding a layer to our neural network**

In [ ]:
class DoubleLayerNN(nn.Module):
    def __init__(self):
        super(DoubleLayerNN, self).__init__()
        # define layers here
        self.fc1 = nn.Linear(784, 100)  # fc => fully connected layer, 28 * 28 = 784
        self.fc2 = nn.Linear(100, 10)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        logit = F.log_softmax(x, dim=1)
        return logit

In [ ]:
mode = DoubleLayerNN()

In [ ]:
model.train()
n_epochs = 2  # change here
for epoch in range(1, n_epochs + 1):
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
# predict all
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for data, target in tqdm(test_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        y_true.extend(target.tolist())
        y_pred.extend(pred.flatten().tolist())

In [ ]:
print("Accuracy", accuracy_score(y_true, y_pred))
print("Precision, Recall, F1-score", precision_recall_fscore_support(y_true, y_pred, average="macro"))

## **Convolutional Neural Network (CCN)**

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model_cnn = CNN()

In [ ]:
model_cnn.train()
n_epochs = 15  # change here
for epoch in range(1, n_epochs + 1):
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        output = model_cnn(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
# predict all
y_true, y_pred = [], []
model_cnn.eval()
with torch.no_grad():
    for data, target in tqdm(test_loader):
        data, target = data.to(device), target.to(device)
        output = model_cnn(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        y_true.extend(target.tolist())
        y_pred.extend(pred.flatten().tolist())

In [ ]:
print("Accuracy", accuracy_score(y_true, y_pred))
print("Precision, Recall, F1-score", precision_recall_fscore_support(y_true, y_pred, average="macro"))